In [79]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk, os, re, string, collections
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from  imageio import imread
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
import multidict as multidict #pip install multidict
import itertools
#import csvtomd
import re
from collections import Counter
#%matplotlib inline
import spacy
from sklearn.preprocessing import MultiLabelBinarizer, Binarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 1. Limpieza y conversión de datos

In [80]:
dtype_test = {"id": np.int32, "keyword": "category"}
test = pd.read_csv("original_data/test.csv", dtype = dtype_test)

In [81]:
dtype_train = {"id": np.int32, "keyword": "category", "target" : bool}
train = pd.read_csv("original_data/train.csv", dtype = dtype_train, encoding='UTF_8')

In [82]:
target_train = train.target

## 1.1 Keywords

In [83]:
nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatize_text(text):
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer  = nltk.stem.WordNetLemmatizer()
    if text == '' or pd.isnull(text):
        return text
    else:
        wordList = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
        s = ' '
        return s.join(wordList)
    
def lemmatize_text_v2(text):
    if text == '' or pd.isnull(text):
        return text
    else:
        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc])

def clean_keyword_field(df):
    df["keyword"] = df["keyword"].str.replace('%20',' ')
    df_sin_keywords_nulos = df.dropna(subset = ['keyword'])
    keywords = np.unique(df_sin_keywords_nulos.keyword)
    for keyword in keywords:
        df.loc[(df['text'].str.contains(keyword, case=False) & df['keyword'].isnull()), 'keyword'] = keyword
    #df['keyword'] = df['keyword'].apply(lemmatize_text_v2)
    return df

## 1.2 Text

### Strict (for tags)

In [84]:
def getStopwordsList():
    fileNamesList = ["texts/99webtools.txt", "texts/atire_ncbi.txt", "texts/atire_puurula.txt", "texts/azure.txt", 
                 "texts/bbalet.txt", 
                 "texts/bow_short.txt", "texts/choi_2000naacl.txt", "texts/cook1988_function_words.txt", 
                 "texts/corenlp_acronym.txt", 
                 "texts/corenlp_hardcoded.txt", "texts/corenlp_stopwords.txt", "texts/datasciencedojo.txt", 
                 "texts/deeplearning4j.txt", 
                 "texts/dkpro.txt", "texts/mongodb.txt", "texts/galago_inquery.txt", "texts/gate_keyphrase.txt", 
                 "texts/gensim.txt", 
                 "texts/glasgow_stop_words.txt", "texts/indri.txt", "texts/kevinbouge.txt", "texts/lexisnexis.txt",
                 "texts/lingpipe.txt", 
                 "texts/mallet.txt", "texts/mysql_innodb.txt", "texts/mysql_myisam.txt", "texts/galago_rmstop.txt", 
                 "texts/atire_ncbi.txt", 
                 "texts/galago_rmstop.txt", "texts/nltk.txt", "texts/okapiframework.txt", "texts/okapi_cacm_expanded.txt", 
                 "texts/onix.txt", 
                 "texts/ovid.txt", "texts/postgresql.txt", "texts/pubmed.txt", "texts/quanteda.txt", "texts/r_tm.txt", 
                 "texts/ranksnl_large.txt", 
                 "texts/reuters_wos.txt", "texts/rouge_155.txt", "texts/scikitlearn.txt", "texts/smart.txt", 
                 "texts/snowball_expanded.txt", 
                 "texts/spacy.txt", "texts/spark_mllib.txt", "texts/sphinx_mirasvit.txt", "texts/t101_minimal.txt", 
                 "texts/taporware.txt", 
                 "texts/terrier.txt", "texts/tonybsk_1.txt", "texts/tonybsk_6.txt", "texts/voyant_taporware.txt", 
                 "texts/weka.txt", 
                 "texts/xapian.txt", "texts/xpo6.txt", "texts/zettair.txt"]
    stopwordsList = []
    for fileName in fileNamesList:
        file = open(fileName, "r", encoding="utf8")
        for line in file:
            stripped_line = line. strip()
            line_list = stripped_line
            if line_list not in stopwordsList:
                stopwordsList.append(line_list)
        file.close()
    return stopwordsList

stopwordsList = getStopwordsList()

print(len(stopwordsList))

2253


In [85]:
def clean_text_strict(text):
    text = text.lower()
    text = text.replace('\d+', '')
    invalid_chars = ['#','|','@','!','?','-','_','[',']','%','&',':','.',',',"''",'/','https','(','//t',')','http',
                 ';','\'']
    for char in invalid_chars:
        if char in text:
            text = text.replace(char,' ')
    #removes url and tags
    text = re.sub(r"(?:\@|https?\://)\S+", " ",  text)
    return text

### Non strict (for special characters, hashtags, etc)

In [86]:
def clean_text_non_strict(text): 
    tw = " "
    words = text.split(' ')
    for word in words:
        word = ''.join(filter(lambda x: x in set(string.printable), word))
        word = word.replace("\n"," ")
        word = word.replace('û',"")
        word = word.replace('Û',"")
        #word = word.replace('_','')
        #word = word.replace("\"",'')
        #word = word.strip('.')
        #word = word.strip(',')
        #word = word.strip(':')
        tw += word + " "
    return tw

## 1.3 Location

In [87]:
#FUNCIONES AUXILIARES: Limpieza de tweet

# Función que saca los links

def quitar_link_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if (("http" not in w) and ("https" not in w)):
            res.append(w)
    return (" ").join(res)

# Función que saca las menciones

def quitar_mencion_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if("@" not in w):
            res.append(w)
    return (" ").join(res)

def quitar_simbolo(tweet, simbolo):
    text = ""
    for c in tweet:
        if(c!=simbolo):
            text+=c
    return text

def agregar_espacio_ente_comas(tweet):
    
    tweer_sin_coma = (" ").join(tweet.split(","))

    return (" ").join(tweer_sin_coma.split("."))

def capitalize_each_word(string):
    
    text = ""
    for w in string.split(" "):
        if(len(w)>3):
            text += w.capitalize()+" "
    return text

def limpiar_tweet_location(tweet):
    res = tweet
    func = [quitar_mencion_twitter, quitar_link_twitter, agregar_espacio_ente_comas, lambda x: quitar_simbolo(x, "#"), capitalize_each_word]
    for f in func:
        res = f(res)
    return res

def to_string(a_list, delimeter):
    res = ""
    
    for e in sorted(filter(None, a_list)):
        res += str(e)+delimeter
    
    if(len(res)==0): return None
    return res[: -len(delimeter)]

In [88]:
# FUNCIONES AUXILIARES: Analisis geografico

from geotext import GeoText #pip install https://github.com/elyase/geotext/archive/master.zip
import pycountry
import geonamescache

gc = geonamescache.GeonamesCache()

def get_country_pycountry(text):
    for country in pycountry.countries:
        # Handle both the cases(Uppercase/Lowercase)
        if str(country.name).lower() in str(text).lower():
            return GeoText(country.name).country_mentions

def get_country_geotext(text):
    return GeoText(text).country_mentions

def get_country(text):
    set1 = set(get_country_geotext(text))
    countries = get_country_pycountry(text)
    if(not countries):
        return list(set1)
    return list(set1.union(set(countries)))

def check_if_US_state(text):
    stateList = []
    states = gc.get_us_states_by_names()
    for state in states:
        if (state in text):
            stateList.append(state)
    return list(set(stateList))

def check_if_US_county(text):
    countyList = []
    stateList = []
    counties = gc.get_us_counties()
    for county in counties:
        name = county.get('name')
        if name in text:
            countyList.append(name)
            stateList.append(county.get('state'))
    return list(set(countyList))

def check_if_continent(text):
    continentList = []
    continents = gc.get_continents()
    for continent,info in continents.items():
        if (info.get('asciiName') in text):
            continentList.append(continent)
    return continentList

def get_US_state(county):
    stateList = []
    counties = gc.get_us_counties()
    if (pd.isnull(county)):
        return county
    for c in counties:
        name = c.get('name')
        if name in county:
            stateList.append(c.get('state'))
    return list(set(stateList))

def get_continent(country):
    continentList = []
    countries = gc.get_countries()
    if (pd.isnull(country)):
        return continentList
    country = country.split(',')
    for c in country:
        info = countries.get(c)
        continentList.append(info.get('continentcode'))
    return list(set(continentList))

### Location in text

In [89]:
# Creo dataframe y limpio la location

location_in_text = train[["id", "text"]].copy()
location_in_text['text'] = location_in_text['text'].apply(lambda x: limpiar_tweet_location(x))

In [90]:
#Creo columnas con informacion
location_in_text["cities"] = location_in_text["text"].apply(lambda x: GeoText(str(x)).cities)
location_in_text["cities_str"] = location_in_text["cities"].apply(lambda x: to_string(x,","))

location_in_text["country"] = location_in_text["text"].apply(lambda x: get_country(x))

location_in_text["counties"] = location_in_text["text"].apply(lambda x: check_if_US_county(str(x)))
location_in_text["counties_str"] = location_in_text["counties"].apply(lambda x: to_string(x,","))

location_in_text["states"] = location_in_text["text"].apply(lambda x: check_if_US_state(str(x)))
location_in_text.loc[(location_in_text['states'].str.len() == 0), 'states'] = location_in_text.counties.apply(lambda x: get_US_state(x))

location_in_text["country_str"] = location_in_text["country"].apply(lambda x: to_string(x,","))
location_in_text["states_str"] = location_in_text["states"].apply(lambda x: to_string(x,","))
location_in_text.loc[~pd.isnull(location_in_text['counties_str']), 'country_str'] = 'US'
location_in_text.loc[~pd.isnull(location_in_text['states_str']), 'country_str'] = 'US'

location_in_text["continent"] = location_in_text["text"].apply(lambda x: check_if_continent(str(x)))
location_in_text.loc[location_in_text['continent'].str.len() == 0, 'continent'] = location_in_text.country_str.apply(lambda x: get_continent(x))

location_in_text["continent_str"] = location_in_text["continent"].apply(lambda x: to_string(x,","))

location_in_text["has_country"] = np.where(pd.isnull(location_in_text['country_str']), False, True)

location_in_text.head()

/home/daniela/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


,id,text,cities,cities_str,country,counties,counties_str,states,country_str,states_str,continent,continent_str,has_country
0,1,Deeds Reason This Earthquake Allah Forgive,[],None,[],[],None,[],None,None,[],None,False
1,4,Forest Fire Near Ronge Sask Canada,[],None,[CA],[],None,[],CA,None,[NA],NA,True
2,5,Residents Asked 'shelter Place' Being Notified...,[],None,[],[],None,[],None,None,[],None,False
3,6,People Receive Wildfires Evacuation Orders Cal...,[],None,[],[],None,[California],US,California,[NA],NA,True
4,7,Just Sent This Photo From Ruby Alaska Smoke Fr...,[],None,[],[],None,[Alaska],US,Alaska,[NA],NA,True


In [91]:
# Selecciono las columnas más útiles para crear features

location_in_text = location_in_text[["id","counties_str","cities_str","states_str","country_str","continent_str","has_country"]]
#location_in_text.columns = ["id", "has_location", "cities", "country"]
location_in_text.head(5)

,id,counties_str,cities_str,states_str,country_str,continent_str,has_country
0,1,None,None,None,None,None,False
1,4,None,None,None,CA,NA,True
2,5,None,None,None,None,None,False
3,6,None,None,California,US,NA,True
4,7,None,None,Alaska,US,NA,True


In [92]:
location_in_text.head()

,id,counties_str,cities_str,states_str,country_str,continent_str,has_country
0,1,None,None,None,None,None,False
1,4,None,None,None,CA,NA,True
2,5,None,None,None,None,None,False
3,6,None,None,California,US,NA,True
4,7,None,None,Alaska,US,NA,True


### Location label info

In [93]:
# Creo dataframe y limpio la location

location = train[["id", "location","target"]].copy().fillna("")
location["location"] = location["location"].apply(lambda x: limpiar_tweet_location(x))

In [94]:
#Creo columnas con informacion

location["has_location"] = location["location"]!=""

#location["cities"] = location["location"].apply(lambda x: GeoText(str(x)).cities)
#location["cities_str"] = location["cities"].apply(lambda x: to_string(x,","))

#location["country"] = location["location"].apply(lambda x: get_country(x))
#location["country_str"] = location["country"].apply(lambda x: to_string(x,","))

location["cities"] = location["location"].apply(lambda x: GeoText(str(x)).cities)
location["cities_str"] = location["cities"].apply(lambda x: to_string(x,","))

location["country"] = location["location"].apply(lambda x: get_country(x))

location["counties"] = location["location"].apply(lambda x: check_if_US_county(str(x)))
location["counties_str"] = location["counties"].apply(lambda x: to_string(x,","))

location["states"] = location["location"].apply(lambda x: check_if_US_state(str(x)))
location.loc[(location['states'].str.len() == 0), 'states'] = location.counties.apply(lambda x: get_US_state(x))

location["country_str"] = location["country"].apply(lambda x: to_string(x,","))
location["states_str"] = location["states"].apply(lambda x: to_string(x,","))
location.loc[~pd.isnull(location['counties_str']), 'country_str'] = 'US'
location.loc[~pd.isnull(location['states_str']), 'country_str'] = 'US'

location["continent"] = location["location"].apply(lambda x: check_if_continent(str(x)))
location.loc[location['continent'].str.len() == 0, 'continent'] = location.country_str.apply(lambda x: get_continent(x))

location["continent_str"] = location["continent"].apply(lambda x: to_string(x,","))

location["has_country"] = np.where(pd.isnull(location['country_str']), False, True)

location[location['has_location']==True].head(50)

/home/daniela/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


,id,location,target,has_location,cities,cities_str,country,counties,counties_str,states,country_str,states_str,continent,continent_str,has_country
31,48,Birmingham,True,True,[Birmingham],Birmingham,[GB],[],None,[],GB,None,[EU],EU,True
32,49,September 2012 Bristol,False,True,[Bristol],Bristol,[GB],[],None,[],GB,None,[EU],EU,True
33,50,Africa,True,True,[],None,[],[],None,[],None,None,[AF],AF,False
34,52,Philadelphia,False,True,[Philadelphia],Philadelphia,[US],[],None,[],US,None,[NA],NA,True
35,53,London,False,True,[London],London,[GB],[],None,[],GB,None,[EU],EU,True
36,54,Pretoria,False,True,[Pretoria],Pretoria,[ZA],[],None,[],ZA,None,[AF],AF,True
37,55,World Wide!!,True,True,[],None,[],[],None,[],None,None,[],None,False
39,57,Paranaque City,False,True,[],None,[],[],None,[],None,None,[],None,False
40,59,Live Webcam,False,True,[],None,[],[],None,[],None,None,[],None,False
42,62,Milky,False,True,[],None,[],[],None,[],None,None,[],None,False


In [95]:
location_aux = location[["country_str", "cities_str", "states_str", "continent_str", "counties_str"]].isnull()[520:525]
location["city_country_not_found"] = location_aux["country_str"] & location_aux["cities_str"] & location_aux["counties_str"] & location_aux["continent_str"] & location_aux["states_str"]

In [96]:
# Selecciono las columnas más útiles para crear features

location_info = location[["id", "has_location", "cities_str", "country_str", "city_country_not_found", "counties_str", "states_str", "continent_str"]]
location_info.columns = ["id", "has_location", "cities", "country", "city_country_not_found", "county", "state", "continent"]

In [97]:
location_info.head()

,id,has_location,cities,country,city_country_not_found,county,state,continent
0,1,False,None,None,NaN,None,None,None
1,4,False,None,None,NaN,None,None,None
2,5,False,None,None,NaN,None,None,None
3,6,False,None,None,NaN,None,None,None
4,7,False,None,None,NaN,None,None,None


## 1.4 Links

In [98]:
links = pd.read_csv("new_data/links.csv", encoding='UTF_8').set_index("id")

In [99]:
import ast

links['http']= links['http'].apply(lambda x: ast.literal_eval(x))

In [100]:
#Funciones auxiliares

def get_dominio(links):
    res= []
    for l in links:
        if(l==None):
            res.append(None)
        else:
            dom = l.split("/")
            res.append(dom[2])
    return res

def type_http(links):
    res = []
    for l in links:
        if(l==None):
            res.append(None)
        else:
            dom = l.split(":")
            res.append(dom[0])
    return res

def count_None(a_list):
    count = 0
    for l in a_list:
        if(l==None):
            count+=1
    return count

def link_encription_generalize(link_encript):
    
    has_http = False
    has_https = False
    
    for type_http in link_encript:
        if type_http==None: 
            continue
        elif type_http== "https":
            has_https = True
        else:
            has_http = True
    
    if(has_http and has_https):
        return "both"
    elif(has_http):
        return "http"
    elif(has_https):
        return "https"
    else:
        return None
    
def to_string(a_list, delimeter):
    res = ""
    
    for e in sorted(filter(None, a_list)):
        res += str(e)+delimeter
    
    if(len(res)==0): return None
    return res[: -len(delimeter)]

In [101]:
links["links_domain"] = links["http"].apply(lambda x: get_dominio(x))
links["links_encription"] = links["http"].apply(lambda x: type_http(x))
links["links_encription_type"] = links["links_encription"].apply(lambda x: link_encription_generalize(x))
links["cant_failed_links"] = links["http"].apply(lambda x: count_None(x))
links["links_domain_str"] = links["links_domain"].apply(lambda x: to_string(x,","))

## 1.5 Tags

### Limpieza de los campos para obtener tags

In [102]:
# FUNCIONES AUXILIARES

def prettify(word_list):
    wl = []
    for word in word_list:
        word = re.sub(r'[^A-Za-z]', " ",  word)
        word = re.sub(r'\b\w\b',' ', word)
        word = re.sub(r'\s+', ' ', word)
        wl.append(word.strip())
    return wl

In [103]:
# Creo el dataframe
df_tags = train[['id','text','target']].copy(deep=True)

# Limpio el dataframe
df_tags['new'] = df_tags["text"].apply(lambda x: clean_text_strict(x))

In [104]:
#  Eliminacion de terminos, caracteres y stopwords innecesarios para generar los tags
#convierte de list a string

df_tags['new'] = df_tags['new'].apply(lambda x: [item for item in x.split() if item not in stopwordsList])
df_tags.new = df_tags.new.apply(prettify)

df_tags.head()

,id,text,target,new
0,1,Our Deeds are the Reason of this #earthquake M...,True,"[deeds, reason, earthquake, allah, forgive]"
1,4,Forest fire near La Ronge Sask. Canada,True,"[forest, fire, ronge, sask, canada]"
2,5,All residents asked to 'shelter in place' are ...,True,"[residents, shelter, notified, officers, evacu..."
3,6,"13,000 people receive #wildfires evacuation or...",True,"[, people, receive, wildfires, evacuation, cal..."
4,7,Just got sent this photo from Ruby #Alaska as ...,True,"[photo, ruby, alaska, smoke, wildfires, pours]"


### Tokenize

In [105]:
# FUNCION AUXILIAR
def lemmatize_text2(word_list):
    text = ' '
    text = text.join(word_list)
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [106]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

df_tags['new_tokenized'] = df_tags.new.apply(lemmatize_text2)
df_tags.new_tokenized = df_tags.new_tokenized.apply(prettify)

df_tags.head()

,id,text,target,new,new_tokenized
0,1,Our Deeds are the Reason of this #earthquake M...,True,"[deeds, reason, earthquake, allah, forgive]","[deed, reason, earthquake, allah, forgive]"
1,4,Forest fire near La Ronge Sask. Canada,True,"[forest, fire, ronge, sask, canada]","[forest, fire, ronge, sask, canada]"
2,5,All residents asked to 'shelter in place' are ...,True,"[residents, shelter, notified, officers, evacu...","[resident, shelter, notified, officer, evacuat..."
3,6,"13,000 people receive #wildfires evacuation or...",True,"[, people, receive, wildfires, evacuation, cal...","[people, receive, wildfire, evacuation, califo..."
4,7,Just got sent this photo from Ruby #Alaska as ...,True,"[photo, ruby, alaska, smoke, wildfires, pours]","[photo, ruby, alaska, smoke, wildfire, pours]"


### Analisis de ocurrencias

In [107]:
def getText(word_list):
    text = ' '
    text = text.join(word_list)
    return text

#metodo para analizar ocurrencias y cantidades
a = df_tags['new_tokenized'].apply(getText).str.lower().str.cat(sep=' ')

words = nltk.tokenize.word_tokenize(a)
word_dist = nltk.FreqDist(words)

this_col = pd.Series(['word','frequency'])

rslt = pd.DataFrame(word_dist.most_common(400),columns = this_col)

In [108]:
frequent_words = rslt ['word']
unique_frequent_words = set(frequent_words)

### Se genera columna con tags a partir de una cantidad de ocurrencias destacadas

In [109]:
def word_finder(x):
    x = set(x)
    return list(unique_frequent_words.intersection(x))

In [110]:
df_tags['tags'] = df_tags.new_tokenized.apply(word_finder)
tags_info = df_tags[['id','target','text','tags']]

tags_info.head()

,id,target,text,tags
0,1,True,Our Deeds are the Reason of this #earthquake M...,"[reason, earthquake]"
1,4,True,Forest fire near La Ronge Sask. Canada,"[forest, fire]"
2,5,True,All residents asked to 'shelter in place' are ...,"[officer, evacuation]"
3,6,True,"13,000 people receive #wildfires evacuation or...","[california, people, wildfire, evacuation]"
4,7,True,Just got sent this photo from Ruby #Alaska as ...,"[smoke, wildfire, photo]"


# 2. Features

## 2.1 Keywords

In [111]:
df_keywords = train.copy()
df_keywords = clean_keyword_field(df_keywords)

In [112]:
df_keywords_lemmatized = df_keywords.copy()
df_keywords_lemmatized.keyword = df_keywords.keyword.apply(lambda x: lemmatize_text_v2(x))

In [113]:
just_links = links.reset_index()[['http','id']]
df_keywords_links = df_keywords.merge(just_links, on='id', how='left')

In [114]:
keywords_lemmatized = getKeywords(df_keywords_lemmatized)
keywords = getKeywords(df_keywords)

def countKeywords(tweet):
    quantity = 0
    tweet = re.sub('[^a-zA-Z\ ]+', "", tweet).lower()
    for word in tweet.split(' '):
        #word = re.sub('[^a-zA-Z]+', '', word)
        if word in keywords:
            quantity+=1
    return quantity

def getHiddenKeywords(tweet):
    k = []
    tweet = re.sub('[^a-zA-Z\ ]+', "", tweet).lower()
    for keyword in keywords:
        if keyword in tweet:
            k.append(lemmatize_text_v2(keyword))
    return k

In [115]:
def toText(listOfWords):
    s=' '
    return s.join(listOfWords)

def textContainsKeyword(row):
    if row.keyword == '' or pd.isnull(row.keyword):
        return False
    else:
        return row.keyword.lower() in row.text.lower()
    
def getKeywords(df):
    df_sin_keywords_nulos = df.dropna(subset = ['keyword'])
    return np.unique(df_sin_keywords_nulos.keyword)

def textContainsKeywordAsHashtag(row):
    hashtags = []
    tweet = row.text.lower()
    if row.keyword == '' or pd.isnull(row.keyword):
        return False
    else:
        for word in tweet.split(' '):
            if (len(word)>1) and (word[0] == '#'):
                hashtags.append(word[1:])
        return row.keyword.lower() in hashtags

def linkContainsKeyword(row):
    links   = row.http
    keyword = row.keyword
    if row.keyword == '' or pd.isnull(row.keyword):
        return False
    if (type(links)==list):
        for link in links:
            if link=='' or pd.isnull(link):
                continue
            if keyword in link:
                return True
    else:
        return False
    return False

def keywords_agregar_features_numericos(df):
    df['text_contains_keyword'] = (df.apply(textContainsKeyword,axis=1)).astype(int)
    df['has_keyword'] = (~df.keyword.isnull()).astype(int)
    df['keywords_quantity'] = df.text.apply(countKeywords)
    df['keywords_mean'] = df.text.apply(countKeywords)/keywords.size
    df['keyword_is_hashtag'] = (df.apply(textContainsKeywordAsHashtag,axis=1)).astype(int)
    df['keyword_frequency'] = df_keywords_lemmatized.groupby('keyword')['id'].transform('count') / keywords_lemmatized.size
    df['link_contains_keyword'] = (df_keywords_links.apply(linkContainsKeyword,axis=1)).astype(int)
    return df

def keywords_agregar_features_categoricos(df):
    df['keywords_in_text'] = df.text.apply(getHiddenKeywords)
    return df

In [116]:
#all_keywords = pd.get_dummies(df_keywords_lemmatized[['id','keyword']],'keyword')
#all_keywords.head()

In [117]:
#df_keywords = df_train.copy()

In [118]:
#df_sin_keywords_nulos = df_keywords.dropna(subset=['keyword'])
#%time df_keywords_not_in_text = df_sin_keywords_nulos[~df_sin_keywords_nulos['text'].str.contains("|".join(df_sin_keywords_nulos['keyword']),case=False)]
#%time df_sin_keywords_nulos[df_sin_keywords_nulos.apply(lambda x: x.keyword in x.text, axis=1)]

In [119]:
keywords_numerical_features = df_keywords.copy()
keywords_numerical_features = keywords_agregar_features_numericos(keywords_numerical_features).drop(columns=['location','text','keyword'])
keywords_numerical_features.head()

,id,target,text_contains_keyword,has_keyword,keywords_quantity,keywords_mean,keyword_is_hashtag,keyword_frequency,link_contains_keyword
0,1,True,1,1,1,0.004525,1,0.227027,0
1,4,True,1,1,1,0.004525,0,0.237838,0
2,5,True,1,1,1,0.004525,0,0.210811,0
3,6,True,1,1,1,0.004525,0,0.210811,0
4,7,True,1,1,1,0.004525,0,0.237838,0


### Encoding de variables categoricas

In [120]:
keywords_categorical_features = df_keywords.copy() 
keywords_categorical_features = keywords_agregar_features_categoricos(keywords_categorical_features).drop(columns=['keyword','location','text'])
keywords_categorical_features.head()

,id,target,keywords_in_text
0,1,True,[earthquake]
1,4,True,"[fire, forest fire]"
2,5,True,[evacuation]
3,6,True,"[evacuation, fire, wildfire]"
4,7,True,"[fire, smoke, wildfire]"


In [121]:
mlb = MultiLabelBinarizer()

keywords_categorical_features = keywords_categorical_features.join(pd.DataFrame(mlb.fit_transform(keywords_categorical_features.pop('keywords_in_text')),
                          columns=mlb.classes_,
                          index=keywords_categorical_features.index).add_prefix('K_'))
keywords_categorical_features

,id,target,K_ablaze,K_accident,K_aftershock,K_airplane accident,K_ambulance,K_annihilate,K_annihilation,K_apocalypse,...,K_volcano,K_war zone,K_weapon,K_whirlwind,K_wild fire,K_wildfire,K_windstorm,K_wound,K_wreck,K_wreckage
0,1,True,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,True,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,True,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,True,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,7,True,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,10869,True,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7609,10870,True,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7610,10871,True,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7611,10872,True,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Agrego features a set de test

In [122]:
test_numerical_keywords = test.copy()
test_categorical_keywords = test.copy()
test_numerical_keywords = keywords_agregar_features_numericos(test_numerical_keywords).drop(columns=['location','text','keyword'])
test_categorical_keywords = keywords_agregar_features_categoricos(test_categorical_keywords).drop(columns=['location','text','keyword'])

In [123]:
test_categorical_keywords = test_categorical_keywords.join(pd.DataFrame(mlb.fit_transform(test_categorical_keywords.pop('keywords_in_text')),
                          columns=mlb.classes_,
                          index=test_categorical_keywords.index).add_prefix('K_'))
test_categorical_keywords.head()

,id,K_ablaze,K_accident,K_aftershock,K_airplane accident,K_ambulance,K_annihilate,K_annihilation,K_apocalypse,K_armageddon,...,K_volcano,K_war zone,K_weapon,K_whirlwind,K_wild fire,K_wildfire,K_windstorm,K_wound,K_wreck,K_wreckage
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Dataframes finales

In [124]:
keywords_numerical_features.to_csv('features/train/features_keywords_numerical.csv', encoding='utf-8')
keywords_categorical_features.to_csv('features/train/features_keywords_in_text_encoded.csv', encoding='utf-8')

In [125]:
test_numerical_keywords.to_csv('features/test/features_keywords_numerical.csv', encoding='utf-8')
test_categorical_keywords.to_csv('features/test/features_keywords_in_text_encoded.csv', encoding='utf-8')

In [126]:
#corrmap = df_keywords.corr()
#fig = plt.figure(figsize=(20,10))

#sns.heatmap(corrmap, vmax=.8, square=True)
#plt.show()

## 2.2 Text

In [148]:
#Funciones auxiliares
stopword = stopwords.words('english')

def stop(text):
    return [w for w in text.split() if w in stopword]

def length(text):
    return(np.mean([len(w) for w in text.split()]))

def punct(text):
    return [c for c in text if c in string.punctuation]

def title(text):
    return [w for w in text.split() if w.istitle()]

def upper_list(text):
    return [w for w in text.split() if w.isupper()]

def lower_list(text):
    return [w for w in text.split() if w.islower()]

def count_special_char(text):
    special_char = 0
    text = text.replace(" ", "")
    for i in range(0, len(text)):
        ch = text[i]
        if (ch.isalpha()):  
            continue
        elif (ch.isdigit()):
            continue
        else:
            special_char += 1
    return special_char

def syllables(text):
    count = 0
    vowels = 'aeiouy'
    for word in text:
        word= word.lower()
        if word[0] in vowels:
            count +=1
        for index in range(1,len(word)):
            if word[index] in vowels and word[index-1] not in vowels:
                count +=1
        if word.endswith('e'):
            count -= 1
        if word.endswith('le'):
            count+=1
        if count == 0:
            count +=1
    return count

def quitar_link_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if (("http" not in w) and ("https" not in w)):
            res.append(w)
    return (" ").join(res)

def quitar_mencion_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if("@" not in w):
            res.append(w)
    return (" ").join(res)

def getHashtags(tweet):
    pat = re.compile(r"#(\w+)")
    return pat.findall(tweet)

def getTaggedUsers(tweet):
    pat = re.compile(r"@(\w+)")
    return pat.findall(tweet)

def agregar_features_cantidad_palabras(df_og):
    df_og['text'] = df_og['text'].apply(lambda x: quitar_link_twitter(x))
    df_og['text'] = df_og['text'].apply(lambda x: quitar_mencion_twitter(x))
    palabras_list = [df_og]
    for df in palabras_list:
        # Número de palabras usadas
        df['#palabras'] = df['text'].apply(lambda x: len(x.split()))
        # Número de palabras únicas
        df['#palabras_unicas'] =  df['text'].apply(lambda x: len(set(x.split())))
        # Número de caracteres
        df['#caracteres'] =  df['text'].apply(lambda x: len(x))
        # Número de stopwords
        df['#stopwords'] = df['text'].apply(lambda x: len(stop(x)))
        # Número de caracteres de puntuación
        df['#puntuacion'] = df['text'].apply(lambda x: len(punct(x)))
        # Número de palabras Capitalizadas
        df['#capitalize'] = df['text'].apply(lambda x: len(title(x)))
        # Número de palabras MAYUSCULAS
        df['#mayusculas'] = df['text'].apply(lambda x: len(upper_list(x)))
        # Número de silabas
        df['#silabas'] = df['text'].apply(lambda x: syllables(x))
        # Promedio de longitud del tweet
        df['promedio_len_word'] = df['text'].apply(lambda x: length(x))
        # Número de caracteres especiales
        df['#caracteres_especiales'] = df['text'].apply(lambda x: count_special_char(x))
    return df

def getUniqueHashtagsList(hashtags):
    hashtagList = []
    for hashtag in hashtags:
        for h in hashtag:
            if h not in hashtagList:
                hashtagList.append(h.lower())
    return hashtagList

def getHashtagsList(hashtags):
    hashtagList = []
    for hashtag in hashtags:
        for h in hashtag:
            hashtagList.append(h.lower())
    return hashtagList

def getMostFrequentHashtag(hashtags):
    maxValue = 0
    finalHashtag = ''
    if (len(hashtags) > 0):
        for hashtag in hashtags:
            hashtag = hashtag.lower()
            if (hashtagDict.get(hashtag) > maxValue):
                finalHashtag = hashtag
                maxValue = hashtagDict.get(hashtag)

    return finalHashtag

def getHashtagFrequency(hashtag):
    if hashtag != '':
        return hashtagDict.get(hashtag)/len(hashtagDict)
    else:
        return 0
    
def getUniqueUsersList(users):
    userList = []
    for user in users:
        for u in user:
            if u not in userList:
                userList.append(u.lower())
    return userList

def getUsersList(users):
    userList = []
    for user in users:
        for u in user:
            userList.append(u.lower())
    return userList

def hasRelevantTaggedUser(users):
    minValue = 5
    #finalUser = ''
    if (len(users) > 0):
        for user in users:
            user = user.lower()
            if (userDict.get(user) > minValue):
                return True
    return False

def text_agregar_features_numericos(df):
    df = agregar_features_cantidad_palabras(df)
    df['#palabras_binned'] = pd.qcut(df['#palabras'], q=4, labels=False)
    df['#palabras_unicas_binned'] = pd.qcut(df['#palabras_unicas'], q=4, labels=False)
    df['#caracteres_binned'] = pd.qcut(df['#caracteres'], q=5, labels=False)
    df['#stopwords_binned']  = pd.qcut(df['#stopwords'], q=3, labels=False)
    df['#puntuacion_binned'] = pd.qcut(df['#puntuacion'], q=3, labels=False)
    df['#capitalize_binned'] = pd.qcut(df['#capitalize'], q=3, labels=False)
    df['#mayusculas_binned'] = pd.qcut(df['#mayusculas'], q=2, labels=False, duplicates='drop')
    df['#silabas_binned']    = pd.qcut(df['#silabas'], q=4, labels=False)
    df['#caracteres_especiales_binned'] = pd.qcut(df['#caracteres_especiales'], q=3, labels=False)
    return df

def hashtags_agregar_features_numericos(df):
    df['hashtags']    = df.text.apply(lambda x: getHashtags(x))
    df['#hashtags']   = df.hashtags.apply(lambda x: len(x))
    df['has_hashtag'] = (df['#hashtags']>0).astype(int)
#    hashtags    = getUniqueHashtagsList(df.hashtags)
#    hashtagDict = Counter(getHashtagsList(df.hashtags))
#    df['relevant_hashtag']      = df.hashtags.apply(lambda x: getMostFrequentHashtag(x))
#    df['freq_relevant_hashtag'] = df.relevant_hashtag.apply(lambda x:getHashtagFrequency(x))
    df = df.drop(columns=['hashtags'])
    return df

def hashtags_agregar_features_categoricos(df):
    df['hashtags'] = df.text.apply(lambda x: getHashtags(x))
    #df_text['relevant_hashtag'] = df_text.hashtags.apply(lambda x: getMostFrequentHashtag(x))
    return df

def tagged_users_agregar_features_numericos(df):
    df['tagged_users'] = df.text.apply(lambda x: getTaggedUsers(x))
    df['#tagged_users'] = df.tagged_users.apply(lambda x: len(x))
#    users = getUniqueUsersList(df.tagged_users)
#    userDict = Counter(getUsersList(df.tagged_users))
#    df['has_tagged_user'] = (df['#tagged_users']>0).astype(int)
#    df['has_relevant_tagged_user'] = (df.tagged_users.apply(lambda x:hasRelevantTaggedUser(x))).astype(int)
    df = df.drop(columns=['tagged_users'])
    return df

def tagged_users_agregar_features_categoricos(df):
    df['tagged_users'] = df.text.apply(lambda x: getTaggedUsers(x))
    return df

In [149]:
df_text = train.copy()
df_text = df_text.drop(columns=['keyword','location'])
df_text.text = df_text.text.apply(clean_text_non_strict)

In [150]:
df_test_text = test.copy()
df_test_text = df_test_text.drop(columns=['keyword','location'])
df_test_text.text = df_test_text.text.apply(clean_text_non_strict)

#### Features TEXT

In [151]:
text_numerical_features = df_text.copy()
text_numerical_features = text_agregar_features_numericos(text_numerical_features).drop(columns=['text'])

In [152]:
test_text_numerical_features = df_test_text.copy()
test_text_numerical_features = text_agregar_features_numericos(test_text_numerical_features).drop(columns=['text'])

#### Features ARROBAS

In [153]:
tagged_users_numerical_features = df_text.copy()
tagged_users_numerical_features = tagged_users_agregar_features_numericos(tagged_users_numerical_features).drop(columns=['text'])
tagged_users_categorical_features = df_text.copy()
tagged_users_categorical_features = tagged_users_agregar_features_categoricos(tagged_users_categorical_features).drop(columns=['text'])

In [154]:
test_tagged_users_numerical_features = df_test_text.copy()
test_tagged_users_numerical_features = tagged_users_agregar_features_numericos(test_tagged_users_numerical_features).drop(columns=['text'])
test_tagged_users_categorical_features = df_test_text.copy()
test_tagged_users_categorical_features = tagged_users_agregar_features_categoricos(test_tagged_users_categorical_features).drop(columns=['text'])
test_tagged_users_numerical_features

,id,#tagged_users
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


TF-IDF - ARROBAS

In [155]:
tagged_users_categorical_features['tagged_users'] = tagged_users_categorical_features['tagged_users'].apply(lambda x:toText(x))
tagged_index = tagged_users_categorical_features['tagged_users'].fillna('').index
tagged_id = tagged_users_categorical_features['id'].fillna('')
corpus = tagged_users_categorical_features['tagged_users'].fillna('')

# Without Smoothing:
tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_tagged=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_tagged.set_index(tagged_index)
df_tf_idf_tagged['id'] = tagged_id

df_tf_idf_tagged.head()

,1061thetwister,115film,15moferadio,1acd4900c1424d1,1rockstar62,20skyhawkmm20,233liveonline,2for1lapdances,4tiles,4tp_kotc,...,zaynmalik,zenande_mcfen,zeno001,zepp1978,zhenghxn,zojadelin,zomatoaus,zourryart,zxathetis,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7


In [156]:
# With Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_smooth_tagged=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_smooth_tagged.set_index(tagged_index)
df_tf_idf_smooth_tagged['id'] = tagged_id

df_tf_idf_smooth_tagged.head()

,1061thetwister,115film,15moferadio,1acd4900c1424d1,1rockstar62,20skyhawkmm20,233liveonline,2for1lapdances,4tiles,4tp_kotc,...,zaynmalik,zenande_mcfen,zeno001,zepp1978,zhenghxn,zojadelin,zomatoaus,zourryart,zxathetis,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7


In [157]:
test_tagged_users_categorical_features['tagged_users'] = test_tagged_users_categorical_features['tagged_users'].apply(lambda x:toText(x))
test_tagged_index = test_tagged_users_categorical_features['tagged_users'].fillna('').index
tagged_id = test_tagged_users_categorical_features['id'].fillna('')
corpus    = test_tagged_users_categorical_features['tagged_users'].fillna('')

# Without Smoothing:
tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
test_tf_idf_tagged=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
test_tf_idf_tagged.set_index(test_tagged_index)
test_tf_idf_tagged['id'] = tagged_id

test_tf_idf_tagged.head()

,1233newcastle,178kakapo,1800_injured,1outside,21joseph,233liveonline,38_degrees,3nach9,40urty,5sos,...,zak_bagans,zakia_kiia,zarnikyaw,zaynmalik,zeenews,zelse007,zerohedge,zjwhitman,zvch4,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11


In [158]:
# With Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
test_tf_idf_smooth_tagged=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
test_tf_idf_smooth_tagged.set_index(test_tagged_index)
test_tf_idf_smooth_tagged['id'] = tagged_id

test_tf_idf_smooth_tagged.head()

,1233newcastle,178kakapo,1800_injured,1outside,21joseph,233liveonline,38_degrees,3nach9,40urty,5sos,...,zak_bagans,zakia_kiia,zarnikyaw,zaynmalik,zeenews,zelse007,zerohedge,zjwhitman,zvch4,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11


#### Features HASHTAGS

In [159]:
hashtags_numerical_features = df_text.copy()
hashtags_numerical_features = hashtags_agregar_features_numericos(hashtags_numerical_features).drop(columns=['text'])
hashtags_categorical_features = df_text.copy()
hashtags_categorical_features = hashtags_agregar_features_categoricos(hashtags_categorical_features).drop(columns=['text'])

In [160]:
test_hashtags_numerical_features = df_test_text.copy()
test_hashtags_numerical_features = hashtags_agregar_features_numericos(test_hashtags_numerical_features).drop(columns=['text'])
test_hashtags_categorical_features = df_test_text.copy()
test_hashtags_categorical_features = hashtags_agregar_features_categoricos(test_hashtags_categorical_features).drop(columns=['text'])

TF-IDF - HASHTAGS

In [161]:
hashtags_categorical_features['hashtags'] = hashtags_categorical_features['hashtags'].apply(lambda x:toText(x))
hashtag_index = hashtags_categorical_features['hashtags'].fillna('').index
hashtag_id = hashtags_categorical_features['id'].fillna('')
corpus = hashtags_categorical_features['hashtags'].fillna('')

# Without Smoothing:
tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_hashtags=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_hashtags.set_index(hashtag_index)
df_tf_idf_hashtags['id'] = hashtag_id

df_tf_idf_hashtags.head()

,0215,034,039,06,09,1008pla_,1008planet_,124,140,16,...,yzf,zaatari,zimmerman,zionism,zionist,ziphimup,zippednews,zippoline,zombiefunrun2014,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7


In [162]:
# With Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_smooth_hashtags=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_smooth_hashtags.set_index(hashtag_index)
df_tf_idf_smooth_hashtags['id'] = hashtag_id

df_tf_idf_smooth_hashtags.head()

,0215,034,039,06,09,1008pla_,1008planet_,124,140,16,...,yzf,zaatari,zimmerman,zionism,zionist,ziphimup,zippednews,zippoline,zombiefunrun2014,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7


In [163]:
test_hashtags_categorical_features['hashtags'] = test_hashtags_categorical_features['hashtags'].apply(lambda x:toText(x))
hashtag_index = test_hashtags_categorical_features['hashtags'].fillna('').index
hashtag_id = test_hashtags_categorical_features['id'].fillna('')
corpus = test_hashtags_categorical_features['hashtags'].fillna('')

# Without Smoothing:
tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
test_df_tf_idf_hashtags=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
test_df_tf_idf_hashtags.set_index(hashtag_index)
test_df_tf_idf_hashtags['id'] = hashtag_id

test_df_tf_idf_hashtags.head()

,039,05,0518,12k,16,1dont,1oak,21dayfix,26,263chat,...,youtube_,ypg,yyc,yychail,yycstorm,yyctraffic,yycweather,zionist,zuma,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11


In [164]:
# With Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
test_df_tf_idf_smooth_hashtags=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
test_df_tf_idf_smooth_hashtags.set_index(hashtag_index)
test_df_tf_idf_smooth_hashtags['id'] = hashtag_id

test_df_tf_idf_smooth_hashtags.head()

,039,05,0518,12k,16,1dont,1oak,21dayfix,26,263chat,...,youtube_,ypg,yyc,yychail,yycstorm,yyctraffic,yycweather,zionist,zuma,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11


Exportacion de Dfs

In [165]:
df_tf_idf_tagged.to_csv('features/train/features_tagged.csv', encoding='utf-8')
df_tf_idf_smooth_tagged.to_csv('features/train/features_tagged_smooth.csv', encoding='utf-8')
df_tf_idf_hashtags.to_csv('features/train/features_hashtags.csv', encoding='utf-8')
df_tf_idf_smooth_hashtags.to_csv('features/train/features_hashtags_smooth.csv', encoding='utf-8')
tagged_users_numerical_features.to_csv('features/train/features_arrobas_numerical.csv', encoding='utf-8')
text_numerical_features.to_csv('features/train/features_text_numerical.csv', encoding='utf-8')
hashtags_numerical_features.to_csv('features/train/features_hashtags_numerical.csv', encoding='utf-8')

In [166]:
test_tf_idf_tagged.to_csv('features/test/features_tagged.csv', encoding='utf-8')
test_tf_idf_smooth_tagged.to_csv('features/test/features_tagged_smooth.csv', encoding='utf-8')
test_df_tf_idf_hashtags.to_csv('features/test/features_hashtags.csv', encoding='utf-8')
test_df_tf_idf_smooth_hashtags.to_csv('features/test/features_hashtags_smooth.csv', encoding='utf-8')
test_tagged_users_numerical_features.to_csv('features/test/features_arrobas_numerical.csv', encoding='utf-8')
test_text_numerical_features.to_csv('features/test/features_text_numerical.csv', encoding='utf-8')
test_hashtags_numerical_features.to_csv('features/test/features_hashtags_numerical.csv', encoding='utf-8')

## 2.3 Location

### Same location in label and in text

In [270]:
#Merge info location in text and in columnn "location"
location_merged = location_in_text.merge(location_info, how="left", on="id",suffixes = ('_in_label', '_in_text'))

In [271]:
location_merged.head()

,id,counties_str,cities_str,states_str,country_str,continent_str,has_country,has_location,cities,country,city_country_not_found,county,state,continent
0,1,None,None,None,None,None,False,False,None,None,NaN,None,None,None
1,4,None,None,None,CA,NA,True,False,None,None,NaN,None,None,None
2,5,None,None,None,None,None,False,False,None,None,NaN,None,None,None
3,6,None,None,California,US,NA,True,False,None,None,NaN,None,None,None
4,7,None,None,Alaska,US,NA,True,False,None,None,NaN,None,None,None


### Location label countries

In [272]:
# Para aquellos tweets con más de dos paises, separo en columnas

countries = location['country_str'].str.split(",", expand = True)

countries["id"] = location["id"]
countries.set_index("id")

countries1 = pd.get_dummies(countries[0],"Country")
countries2 = pd.get_dummies(countries[1],"Country")

# Uno las columnas
for country in countries1.columns:
    if(country in countries2.columns):
        countries1[country] = countries1[country] + countries2[country]
for country in countries2.columns:
    if(country in countries1.columns):
        countries1[country] = countries2[country]

In [273]:
countries1.head(2)

,Country_AE,Country_AF,Country_AQ,Country_AR,Country_AT,Country_AU,Country_BB,Country_BD,Country_BE,Country_BG,...,Country_TH,Country_TM,Country_UA,Country_UG,Country_US,Country_UY,Country_VE,Country_VN,Country_ZA,Country_ZW
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### BOW: Coutry

In [274]:
#bow_country_index = location['country_str'].fillna('').index
corpus = list(location['country_str'].fillna(''))

In [275]:
country_vectorizer = CountVectorizer()

bow_country = country_vectorizer.fit_transform(corpus).todense()
df_bow_country = pd.DataFrame(bow_country)
df_bow_country.set_index(location['id'].index)
df_bow_country.columns = country_vectorizer.get_feature_names()

In [276]:
df_bow_country.head()

,ae,af,aq,ar,at,au,bb,bd,be,bg,...,th,tm,ua,ug,us,uy,ve,vn,za,zw
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Tf-Idf: Country

In [277]:
# Without Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_country=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_country.set_index(location['id'].index)

df_tf_idf_country.head()

,ae,af,aq,ar,au,bb,bd,bg,bh,br,...,sy,th,tm,ua,ug,uy,ve,vn,za,zw
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [278]:
# With Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_smooth_country=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_smooth_country.set_index(location['id'].index)

df_tf_idf_smooth_country.head()

,ae,af,aq,ar,au,bb,bd,bg,bh,br,...,sy,th,tm,ua,ug,uy,ve,vn,za,zw
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [279]:
nueva_posicion= df_bow_country.columns.get_loc('ae')
df_bow_country.insert(loc= nueva_posicion, column='index', value = df_bow_country.index)
df_bow_country

,index,ae,af,aq,ar,at,au,bb,bd,be,...,th,tm,ua,ug,us,uy,ve,vn,za,zw
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,7608,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7609,7609,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7610,7610,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7611,7611,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [280]:
df_tf_idf_smooth_country.insert(loc= nueva_posicion, column='index', value = df_tf_idf_smooth_country.index)
df_tf_idf_smooth_country

,index,ae,af,aq,ar,au,bb,bd,bg,bh,...,sy,th,tm,ua,ug,uy,ve,vn,za,zw
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,7608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7609,7609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7610,7610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7611,7611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_tf_idf_smooth_country

In [ ]:
#df_tf_idf_smooth_country.to_csv('features/features_location_tf_idf.csv', encoding='utf-8')
#df_bow_country.to_csv('features/features_location_bow.csv', encoding='utf-8')

## 2.4 Links

#### Features con cantidades

In [71]:
links_info = links[["links_cant", "cant_failed_links", "links_encription_type"]].copy()

In [72]:
links_info.head()

,links_cant,cant_failed_links,links_encription_type
id,,,
48,1,0,https
49,1,0,https
50,1,0,https
53,1,0,https
55,1,0,https


#### Features con cantidad de apariciones de n dominios más usuales

In [73]:
# n cantidad de links
n_most_common_links = 20

In [74]:
# Dejo en cada columna un links

links_column = links['links_domain_str'].str.split(",", expand = True)
links_column[105:110]

,0,1,2,3
id,,,,
314,retrospring.net,None,None,None
316,twitter.com,None,None,None
322,twitter.com,www.alaraby.co.uk,None,None
338,t.co,twitter.com,None,None
340,syrianperspective.com,None,None,None


In [75]:
links_cant_domain = pd.get_dummies(links_column[0],"domain")

In [76]:
# Sumo las columnas de acuerdo al link

unique_domains = set(links_cant_domain.columns)

for i in range(1, len(links_column.columns)):
    n_links_pd = pd.get_dummies(links_column[i],"domain")
    for domain in unique_domains:
        if(domain in set(n_links_pd.columns)):
            links_cant_domain[domain] = links_cant_domain[domain] + n_links_pd[domain]

In [77]:
links_cant_domain.head()

,domain_1.usa.gov,domain_24x7freeadvice.wordpress.com,domain_350.org,domain_99u.adobe.com,domain_9jacruz.blogspot.com,domain_9jalatestsnews.blogspot.com,domain_a.msn.com,domain_abc11.com,domain_abc7.com,domain_abc7chicago.com,...,domain_www.zerohedge.com,domain_www.ziprecruiter.com,domain_www.zoma.to,domain_www1.cbn.com,domain_www2.conning.com,domain_www2.deloitte.com,domain_yidwithlid.blogspot.com,domain_yonews.org,domain_zeenews.india.com,domain_zenit.org
id,,,,,,,,,,,,,,,,,,,,,
48,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
# n most common domains
n_domains = 20

In [79]:
# calculo los dominios más usuales
unique_domains_ordered = links_column[0].value_counts().index
columns_n_unique_links = ["domain_"+link for link in unique_domains_ordered[:n_domains]]

most_common_domain_cant = links_cant_domain[columns_n_unique_links]

In [80]:
most_common_domain_cant.head()

,domain_twitter.com,domain_www.youtube.com,domain_news.google.com,domain_t.co,domain_www.facebook.com,domain_www.instagram.com,domain_www.bbc.co.uk,domain_abcnews.go.com,domain_www.latimes.com,domain_cue.li,domain_www.amazon.com,domain_feedsportal.com?utm_source=twitterfeed&utm_medium=twitter&utm_campaign=Feed%3A+ndtv%2FTqgX+%28NDTV+News+-+World%29,domain_earthquake.usgs.gov,domain_edition.cnn.com,domain_www.dailymail.co.uk,domain_fanarmybracket2018.microsites.bbthr.com,domain_www.theguardian.com,domain_www.google.com,domain_news.yahoo.com,domain_www.wired.com
id,,,,,,,,,,,,,,,,,,,,
48,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### BOW: Dominios

In [81]:
bow_domain_index = links['links_domain_str'].dropna().index
corpus = list(links['links_domain_str'].dropna())


In [82]:
domain_vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

bow_domain = domain_vectorizer.fit_transform(corpus).todense()
df_bow_domain = pd.DataFrame(bow_domain)
df_bow_domain.set_index(bow_domain_index)
df_bow_domain.columns = domain_vectorizer.get_feature_names()

In [83]:
df_bow_domain.head()

,10news,1stheadlines,24x7freeadvice,28chronicle,28national,28ndtv,28zero,29,2c,2ffeed,...,zalkus,zamanalwsl,zeenews,zenit,zeppy,zero,zerohedge,ziprecruiter,zoma,zw
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Tf-Idf:  Domain

In [84]:
bow_domain_index = links['links_domain_str'].dropna().index
corpus = list(links['links_domain_str'].dropna())

In [85]:
# Without Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_domain=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_domain.set_index(bow_domain_index)

df_tf_idf_domain.head()

,10news,1stheadlines,24x7freeadvice,28chronicle,28national,28ndtv,28zero,29,2c,2ffeed,...,zalkus,zamanalwsl,zeenews,zenit,zeppy,zero,zerohedge,ziprecruiter,zoma,zw
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
# With Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_smooth_domain=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_smooth_domain.set_index(bow_domain_index)

df_tf_idf_smooth_domain.head()

,10news,1stheadlines,24x7freeadvice,28chronicle,28national,28ndtv,28zero,29,2c,2ffeed,...,zalkus,zamanalwsl,zeenews,zenit,zeppy,zero,zerohedge,ziprecruiter,zoma,zw
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.5 Tags

### Informacion general tags

In [254]:
tags_info = tags_info.copy()
tags_info["cant_tags"] = tags_info["tags"].apply(lambda x: len(x))
tags_info["tags_str"] = tags_info["tags"].apply(lambda x: to_string(x,","))

In [255]:
tags_info.head(2)

,id,target,text,tags,cant_tags,tags_str
0,1,True,Our Deeds are the Reason of this #earthquake M...,"[earthquake, reason]",2,"earthquake,reason"
1,4,True,Forest fire near La Ronge Sask. Canada,"[forest, fire]",2,"fire,forest"


###  Nuevo feature por tag

In [256]:
tags_columns = tags_info['tags_str'].str.split(",", expand = True)

In [257]:

df_has_tags = pd.get_dummies(tags_columns[0],"tag")
unique_tag = set(df_has_tags.columns)

for i in range(1, len(tags_columns.columns)):
    n_links_pd = pd.get_dummies(tags_columns[i],"tag")
    for domain in unique_tag:
        if(domain in set(tags_columns.columns)):
            df_has_tags[domain] = df_has_tags[domain] + tags_columns[domain]

In [258]:
df_has_tags["id"] = tags_info["id"]
df_has_tags = df_has_tags.set_index("id")

In [259]:
df_has_tags.head()

,tag_abc,tag_ablaze,tag_accident,tag_action,tag_air,tag_aircraft,tag_airplane,tag_airport,tag_ambulance,tag_america,...,tag_wildfire,tag_win,tag_wind,tag_windstorm,tag_woman,tag_wound,tag_wounded,tag_wreck,tag_wrecked,tag_zone
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### BOW: tags

In [260]:
#bow_tags_index = tags_info['tags_str'].fillna('').index
corpus = list(tags_info['tags_str'].fillna(''))

In [261]:
domain_vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

bow_tags = domain_vectorizer.fit_transform(corpus).todense()
df_bow_tags = pd.DataFrame(bow_tags)
df_bow_tags.set_index(tags_info['id'].index)
df_bow_tags.columns = domain_vectorizer.get_feature_names()

In [262]:
df_bow_tags.head()

,abc,ablaze,accident,action,air,aircraft,airplane,airport,ambulance,america,...,wind,windstorm,woman,wound,wounded,wreck,wreckage,wrecked,yr,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Tf-idf: tags

In [263]:
#bow_tags_index = tags_info['tags_str'].dropna().index
corpus = list(tags_info['tags_str'].fillna(''))

In [264]:
# Without Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_tags=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_bow_tags.set_index(tags_info['id'].index)

df_tf_idf_tags.head()

,abc,ablaze,accident,action,air,aircraft,airplane,airport,ambulance,america,...,wind,windstorm,woman,wound,wounded,wreck,wreckage,wrecked,yr,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [265]:
# With Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_smooth_tags =pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_bow_tags.set_index(tags_info['id'].index)

df_tf_idf_smooth_tags.head()

,abc,ablaze,accident,action,air,aircraft,airplane,airport,ambulance,america,...,wind,windstorm,woman,wound,wounded,wreck,wreckage,wrecked,yr,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [281]:
nueva_posicion= df_bow_tags.columns.get_loc('abc')
df_bow_tags.insert(loc= nueva_posicion, column='index', value = df_bow_tags.index)
df_bow_tags

,index,abc,ablaze,accident,action,air,aircraft,airplane,airport,ambulance,...,wind,windstorm,woman,wound,wounded,wreck,wreckage,wrecked,yr,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,7608,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7609,7609,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7610,7610,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7611,7611,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [282]:
df_tf_idf_smooth_tags.insert(loc= nueva_posicion, column='index', value = df_tf_idf_smooth_tags.index)
df_tf_idf_smooth_tags

,index,abc,ablaze,accident,action,air,aircraft,airplane,airport,ambulance,...,wind,windstorm,woman,wound,wounded,wreck,wreckage,wrecked,yr,zone
0,0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,7608,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7609,7609,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7610,7610,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7611,7611,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#df_tf_idf_smooth_tags.to_csv('features/features_tags_tf_idf.csv', encoding='utf-8')
#df_bow_tags.to_csv('features/features_tags_bow.csv', encoding='utf-8')